In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
def get_apparitions( url ):
    html = requests.get(url)
    bs_obj = BeautifulSoup( html.text, "lxml")
    h4 = bs_obj.find({"h4":"Aparições em títulos da série:"})
    apparitions = [ i.text.replace("/", "-") for i in h4.findNext().find_all("a") ]
    return apparitions

def get_persona( persona_url ):
    apparitions = get_apparitions( persona_url )
    persona = persona_url.split("/")[-2]
    dct = { "persona":[persona]*len(apparitions),
            "apparitions":apparitions}
    df = pd.DataFrame(dct)
    return df

def get_personas_urls( url ):
    html = requests.get( url )
    bs_obj = BeautifulSoup( html.text, "lxml" )
    h3s = bs_obj.find_all( "h3", style="padding-left: 30px;" )
    personas_urls = []
    for i in h3s:
        personas_urls += [ j.attrs["href"] for j in i.findNext().find_all("a") ]
    return personas_urls

def get_all_personas( ):
    url = "http://www.residentevildatabase.com/personagens/"
    personas_urls = get_personas_urls( url )
    dfs = [ get_persona(i) for i in personas_urls ]
    df = pd.concat( dfs, ignore_index=True )
    return df    

### Getting data from website (residentevildatabase.com)

In [3]:
url = "http://www.residentevildatabase.com/personagens/"
df = get_all_personas()

### Transforming data to transacional dataset

In [4]:
def sort_names( row ):
    'Method to drop same relationship, i.e., persona_1 -> persona_2, persona_2 -> persona_1'
    names = [ row["persona_1"], row["persona_2"] ]
    names.sort()
    names = ", ".join(names)
    return names

In [5]:
df_analysis = pd.merge( left=df, right=df,
                        how="left", on="apparitions",
                        suffixes=("_1", "_2") )

df_analysis = df_analysis[ df_analysis["persona_1"] != df_analysis["persona_2"] ]

df_analysis["names"] = df_analysis.apply(sort_names, axis=1)
df_analysis = df_analysis.drop_duplicates(subset=["names", "apparitions"])

df_analysis = df_analysis[["persona_1", "persona_2", "apparitions"]]

df_analysis.head()

,persona_1,persona_2,apparitions
1,ada-wong,annette-birkin,Biohazard - Resident Evil 2
2,ada-wong,ben-bertolucci,Biohazard - Resident Evil 2
3,ada-wong,brian-irons,Biohazard - Resident Evil 2
4,ada-wong,claire-redfield,Biohazard - Resident Evil 2
5,ada-wong,hunk,Biohazard - Resident Evil 2
